## 📗 Prompt template Basics 📗 ##

In [6]:
from typing import Dict, List

from langchain_community.llms import Ollama
llm = Ollama(model="llama3",temperature=0.6)

### 📄 Prompt templates using from_template method 📄

In [40]:
from langchain_core.prompts import PromptTemplate

# 1st way of creating a prompt template. 
# using "from_template" method of PromptTemplate class.
pt = PromptTemplate.from_template('tell me a funny joke about {content}')     # --> returns an object of PromptTemplate class.


# we can format it just like string formatting.
print(pt.format(content = 'elephant'))                  # --> prints prompt as string.


# we can also invoke a pt object as it is runnable.
final_prompt = pt.invoke({"content":"elephant"})        
final_prompt                                            # --> returns StringPromptValue instance
print(final_prompt)                                     # --> prints a string variable with prompt as its value.                  

tell me a funny joke about elephant
text='tell me a funny joke about elephant'


### 🛠️ prompt templates using constructors 🛠️

In [8]:
# using PromptTemplate class, using PromptTemplate class constructor directly.

pt = PromptTemplate(
    template='Tell me a {adjective} joke about {content}',
    input_variables=['adjective', 'content']
)

print(pt.format(adjective='good',content= 'elephant'))             # --> prints prompt as string.

final_prompt = pt.invoke({"adjective" : "funny", "content" : "elephant"})
final_prompt                                                        # --> returns StringPromptValue instance
print(final_prompt)                                                 # --> prints a string variable with prompt as its value.


Tell me a good joke about elephant
text='Tell me a funny joke about elephant'


### 🦙 Send prompt to LLM 🦙

In [9]:
# > we can send the prompt to llm by two ways. <#
# >   1. Directly invoking the llm
# >   2. Creating a chain of two objects.


# 1st is directly invoking the llm object.
# llm.invoke(final_prompt)                            # --> returns a non parsed string output.


# 2nd way we can chain prompt and llm object. chain direction is left --> right.
chain = pt | llm
chain.invoke(
    {"adjective": "new", "content": "elephant"}
)  # --> returns a non parsed string output.

# Second way with chaining is preferred !

'Why did the elephant quit the circus?\n\nBecause it was tired of working for peanuts and wanted to trunk its own business!'

### 📖 few shot example prompt templates 📖

In [62]:
from langchain_core.prompts import FewShotPromptTemplate

# first create a list of dictionaries as examples.
user_provided_examples: List[Dict[str, str]] = [
    {
        "question": "what is 2 c 2 ?",
        "answer": "4",
    },
    {
        "question": "what is 5 c 3 ?",
        "answer": "15",
    },
    {
        "question": "what is 12 c 2 ?",
        "answer": "24",
    },
]


# then we will create a template showing how to present examples to the LLM.
example_template = """
    Question : {question}
    Answer : {answer}
"""


# we will use that template to create a prompt_template
example_pt = PromptTemplate(
    template=example_template,
    input_variables=["question", "answer"],
)

# Checking the format of our examples prompt template.
example_pt.invoke(user_provided_examples[0]).to_string()


system_message = ''' A Human is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided that is suitable for the human. Try to learn from these examples and answer the final question asked by the user. Examples are - '''



# Then we'll create a few shots prompt template using our examples and example_prompt_template
fewshot_pt = FewShotPromptTemplate(
    examples = user_provided_examples,
    example_prompt = example_pt,

    prefix = system_message,                # We can also pass a system message to using prefix.
    suffix="Question: {question}",          # suffix is a prompt_template for final question, which is not an example.
    input_variables=["question"],
)


# Finally we'll invoke the fewshot_pt to create final prompt that will be sent to the llm.
final_prompt = fewshot_pt.invoke({"question": "what is 10 c 2 ?"})
print(final_prompt)

text=' A Human is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided that is suitable for the human. Try to learn from these examples and answer the final question asked by the user. Examples are - \n\n\n    Question : what is 2 c 2 ?\n    Answer : 4\n\n\n\n    Question : what is 5 c 3 ?\n    Answer : 15\n\n\n\n    Question : what is 12 c 2 ?\n    Answer : 24\n\n\nQuestion: what is 10 c 2 ?'


### ⛓️‍💥 Chaining all the parts together ⛓️‍💥

In [63]:
from langchain_core.output_parsers import StrOutputParser

# use String parser to get output in string 
str_parser = StrOutputParser()


chain = fewshot_pt | llm | str_parser
# chain.invoke({"question": "In the above example please explain what does m stands for and what is 12 m 12 ?"})

### 🦙 Stream the FewShotExample prompt result from LLM 🦙

In [64]:
line = ''

# If we want to stream output line by line.
for chunk in chain.stream({"question": "what is 12 c 12 ?"}):
    if chunk == '.' or chunk == '\n':
        line += chunk
        print(line)
        line = ""
    
    else:
        line += chunk

print(line)

I think I've learned something new!

Based on the examples, it seems that "c" represents multiplication.
 Therefore, if we apply this to the final question:

What is 12 c 12?

Answer: 144


### 📖 FewShotExamples with example selection 📖

In [66]:

#? from langchain_core.prompts import FewShotPromptTemplate

from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# first create an extensive list of dictionaries as examples.
user_provided_examples: List[Dict[str, str]] = [
    {
        "question": "what is 2 c 2 ?",
        "answer": "4",
    },
    {
        "question": "what is 5 c 3 ?",
        "answer": "15",
    },
    {
        "question": "what is 12 c 2 ?",
        "answer": "24",
    },
    {
        "question": "what is 5 m 2 ?",
        "answer": "2.5",
    },
    {
        "question": "what is 12 m 3 ?",
        "answer": "4",
    },
    {
        "question": "what is 15 m 15 ?",
        "answer": "1",
    },
]


# Create an example selector 
example_selector = SemanticSimilarityExampleSelector.from_examples(
    user_provided_examples,                 # This is the list of examples available to select from.
    OpenAIEmbeddings(),                     # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    Chroma,                                 # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    k=2,                                    # This is the number of examples to produce.
)



# Select the most similar example to the input.
question = "what is 10 c 2 ?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")



# # then we will create a template showing how to present examples to the LLM.
# example_template = """
#     Question : {question}
#     Answer : {answer}
# """


# # we will use that template to create a prompt_template
# example_pt = PromptTemplate(
#     template=example_template,
#     input_variables=["question", "answer"],
# )

# # Checking the format of our examples prompt template.
# example_pt.invoke(user_provided_examples[0]).to_string()


# system_message = ''' A Human is trying to teach you something new by giving examples below in the form of question and answers. For every question there is an answer provided that is suitable for the human. Try to learn from these examples and answer the final question asked by the user. Examples are - '''



# # Then we'll create a few shots prompt template using our examples and example_prompt_template
# fewshot_pt = FewShotPromptTemplate(
#     examples = user_provided_examples,
#     example_prompt = example_pt,

#     prefix = system_message,                # We can also pass a system message to using prefix.
#     suffix="Question: {question}",          # suffix is a prompt_template for final question, which is not an example.
#     input_variables=["question"],
# )


# # Finally we'll invoke the fewshot_pt to create final prompt that will be sent to the llm.
# final_prompt = fewshot_pt.invoke({"question": "what is 10 c 2 ?"})
# print(final_prompt)

ModuleNotFoundError: No module named 'langchain_openai'